A Technical tutor that uses GPT-4o-mini and ollama to answer your questions

In [24]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import ollama


In [25]:
load_dotenv(override=True)
api_key=os.getenv("OPENAI_API_KEY")
MODEL='gpt-4o-mini'
openai=OpenAI()


In [26]:
def ask_gpt4(question):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": question}], stream=True
    )
    for chunk in response:
        if chunk.choices and chunk.choices[0].delta.content:
            yield chunk.choices[0].delta.content


def ask_ollama(question):
    response = ollama.chat(model="llama3.2", messages=[{"role": "user", "content": question}], stream=True)

    for chunk in response:
        if "message" in chunk and "content" in chunk["message"]:
            yield chunk["message"]["content"] 


In [27]:
import ipywidgets as widgets
from IPython.display import display


# Dropdown to select model
model_selector = widgets.Dropdown(
    options=["GPT-4o-mini", "Ollama"],
    value="GPT-4o-mini",
    description="Model:"
)

# Text input for user questions
question_input = widgets.Text(
    placeholder="Type your programming question..."
)

# Button to submit
submit_button = widgets.Button(description="Ask")

# Output widget
output = widgets.Output()

def on_submit(b):
    with output:
        output.clear_output()
        question = question_input.value
        model = model_selector.value

        if model == "GPT-4o-mini":
            generator = ask_gpt4(question)
        else:
            generator = ask_ollama(question)

        response_text = ""
        for chunk in generator:
            response_text += chunk
            output.clear_output(wait=True)  # Update dynamically
            print(response_text, end="", flush=True)

# Attach event to button
submit_button.on_click(on_submit)

# Display widgets
display(model_selector, question_input, submit_button, output)

Dropdown(description='Model:', options=('GPT-4o-mini', 'Ollama'), value='GPT-4o-mini')

Text(value='', placeholder='Type your programming question...')

Button(description='Ask', style=ButtonStyle())

Output()